In [87]:
import copy
import io
import numpy as np
import py3Dmol
from Bio.PDB import PDBParser, PDBIO
import numpy as np
import time
import plotly.graph_objects as go
import os
from tqdm import tqdm

import torch
import nglview as nv
import py3Dmol
from openmm.app import PDBFile, ForceField, Simulation, NoCutoff
from openmm import Context, VerletIntegrator, Platform
from openmm.unit import nanometer, kilojoule_per_mole, picoseconds, Quantity
import numpy as np
from Bio.PDB import PDBParser
import plotly.graph_objects as go

In [88]:
# ---------- Utility Functions ----------

def calc_dihedral(p0, p1, p2, p3):
    """
    Calculate the dihedral angle (in degrees) defined by four points.
    """
    b0 = p0 - p1
    b1 = p2 - p1
    b2 = p3 - p2

    b1 /= np.linalg.norm(b1)
    v = b0 - np.dot(b0, b1) * b1
    w = b2 - np.dot(b2, b1) * b1

    x = np.dot(v, w)
    y = np.dot(np.cross(b1, v), w)
    return np.degrees(np.arctan2(y, x))

def rotation_matrix(axis, theta):
    """
    Return a rotation matrix that rotates by theta (in radians) about the given axis.
    """
    axis = axis / np.linalg.norm(axis)
    cos_t = np.cos(theta)
    sin_t = np.sin(theta)
    one_minus_cos = 1 - cos_t
    x, y, z = axis
    R = np.array([
        [cos_t + x * x * one_minus_cos,
         x * y * one_minus_cos - z * sin_t,
         x * z * one_minus_cos + y * sin_t],
        [y * x * one_minus_cos + z * sin_t,
         cos_t + y * y * one_minus_cos,
         y * z * one_minus_cos - x * sin_t],
        [z * x * one_minus_cos - y * sin_t,
         z * y * one_minus_cos + x * sin_t,
         cos_t + z * z * one_minus_cos]
    ])
    return R

# ---------- Dihedral Adjustment Functions ----------

def set_phi_dihedral(structure, new_phi):
    """
    Adjust the φ (phi) dihedral for residue 2 (ALA) in chain X.
    
    φ is defined by:
      C (residue 1, ACE) - N (residue 2, ALA) - CA (residue 2, ALA) - C (residue 2, ALA)
    
    Rotation is applied about the N–CA bond.
    We hold atoms upstream (including the N) fixed and rotate all atoms attached
    to CA that are "downstream": in residue 2 (all except N and CA) and in residue 3.
    """
    model = structure[0]
    chain = model["X"]

    # Identify the four atoms that define φ.
    atom_ids = [
        ("X", 1, "C"),   # C from ACE (residue 1)
        ("X", 2, "N"),   # N from ALA (residue 2)
        ("X", 2, "CA"),  # CA from ALA (residue 2)
        ("X", 2, "C")    # C from ALA (residue 2)
    ]
    atoms = []
    for ch, resnum, atom_name in atom_ids:
        atoms.append(model[ch][resnum][atom_name])
    p0, p1, p2, p3 = [atom.get_coord() for atom in atoms]
    
    print(f"indices for phi: {atom_ids}")

    current_phi = calc_dihedral(p0, p1, p2, p3)
    delta = np.radians(new_phi - current_phi)
    print(f"Current φ: {current_phi:.2f}°; rotating by {np.degrees(delta):.2f}° to {new_phi:.2f}°")

    # Rotation axis is the N–CA bond.
    axis = p2 - p1
    axis /= np.linalg.norm(axis)
    R = rotation_matrix(axis, delta)
    pivot = p2  # Use CA as the pivot.

    # Define moving atoms:
    # In residue 2 (ALA): all atoms except "N" and "CA"
    # In residue 3 (NME): all atoms.
    moving_atoms = []
    res2 = chain[2]
    for atom in res2:
        if atom.get_id() not in ["N", "CA"]:
            moving_atoms.append(atom)
    if 3 in chain:
        res3 = chain[3]
        for atom in res3:
            moving_atoms.append(atom)

    # Apply rotation to moving atoms.
    for atom in moving_atoms:
        pos = atom.get_coord() - pivot
        new_pos = np.dot(R, pos) + pivot
        atom.set_coord(new_pos)

def set_psi_dihedral(structure, new_psi):
    """
    Adjust the ψ (psi) dihedral for residue 2 (ALA) in chain X.
    
    ψ is defined by:
      N (residue 2, ALA) - CA (residue 2, ALA) - C (residue 2, ALA) - N (residue 3, NME)
    
    Rotation is applied about the CA–C bond.
    We hold CA and C fixed and rotate all atoms attached to C that are "downstream".
    """
    model = structure[0]
    chain = model["X"]

    atom_ids = [
        ("X", 2, "N"),    # N from ALA (residue 2)
        ("X", 2, "CA"),   # CA from ALA (residue 2)
        ("X", 2, "C"),    # C from ALA (residue 2)
        ("X", 3, "N")     # N from NME (residue 3)
    ]
    atoms = []
    for ch, resnum, atom_name in atom_ids:
        atoms.append(model[ch][resnum][atom_name])
    p0, p1, p2, p3 = [atom.get_coord() for atom in atoms]
    
    print(f"indices for psi: {atom_ids}")
    
    current_psi = calc_dihedral(p0, p1, p2, p3)
    delta = np.radians(new_psi - current_psi)
    print(f"Current ψ: {current_psi:.2f}°; rotating by {np.degrees(delta):.2f}° to {new_psi:.2f}°")

    # Rotation axis is the CA–C bond.
    axis = p2 - p1  # p1 = CA, p2 = C.
    axis /= np.linalg.norm(axis)
    R = rotation_matrix(axis, delta)
    pivot = p2  # Use C as the pivot.

    # Define moving atoms:
    # In residue 2 (ALA): all atoms except "N", "CA", and "C"
    # In residue 3 (NME): all atoms.
    moving_atoms = []
    res2 = chain[2]
    for atom in res2:
        if atom.get_id() not in ["N", "CA", "C"]:
            moving_atoms.append(atom)
    if 3 in chain:
        res3 = chain[3]
        for atom in res3:
            moving_atoms.append(atom)

    # Apply the rotation.
    for atom in moving_atoms:
        pos = atom.get_coord() - pivot
        new_pos = np.dot(R, pos) + pivot
        atom.set_coord(new_pos)



In [89]:


def pdb_to_xyz(pdbfile):
    """Convert PDB file to xyz coordinates, atom types, and atom info"""
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('protein', pdbfile)
    
    # Get coordinates, atom types and atom info
    coords = []
    atom_types = []
    atom_info = []
    
    for model in structure:
        for chain in model:
            for residue in chain:
                for atom in residue:
                    coords.append(atom.get_coord())
                    atom_types.append(atom.element)
                    atom_info.append(f"{atom.get_name()} from {residue.get_resname()} (residue {residue.get_id()[1]})")
                    
    coords = np.array(coords)
    
    return coords, atom_types, atom_info

def find_bonds(coords, atom_types, max_dist=2.0):
    """Find bonds between atoms based on distance criteria"""
    bonds = []
    n_atoms = len(coords)
    
    # Typical bond lengths (in Angstroms)
    bond_lengths = {
        ('C', 'C'): 1.54,
        ('C', 'N'): 1.47,
        ('C', 'O'): 1.43,
        ('C', 'H'): 1.09,
        ('N', 'H'): 1.01,
        ('O', 'H'): 0.96
    }
    
    # Make bond lengths symmetric
    for (a1, a2), dist in list(bond_lengths.items()):
        bond_lengths[(a2, a1)] = dist
    
    # Calculate distances between all pairs of atoms
    for i in range(n_atoms):
        for j in range(i+1, n_atoms):
            # Skip if both atoms are hydrogen
            if atom_types[i] == 'H' and atom_types[j] == 'H':
                continue
                
            dist = np.linalg.norm(coords[i] - coords[j])
            
            # Get expected bond length
            expected_length = bond_lengths.get((atom_types[i], atom_types[j]), 1.5)
            
            # Add bond if distance is within tolerance of expected length
            if dist < expected_length * 1.3:  # 30% tolerance
                bonds.append((i, j))
                
    return bonds

# Convert PDB to xyz coordinates
coords, atom_types, atom_info = pdb_to_xyz("alanine_dipeptide_nowater.pdb")

# Find bonds
bonds = find_bonds(coords, atom_types)

# Save coordinates to xyz file
with open('alanine_dipeptide.xyz', 'w') as f:
    f.write(f"{len(coords)}\n")
    f.write("Alanine dipeptide structure\n")
    for atom_type, coord in zip(atom_types, coords):
        f.write(f"{atom_type:2s} {coord[0]:10.6f} {coord[1]:10.6f} {coord[2]:10.6f}\n")



In [92]:
def structure_to_pdb_string(structure):
    """Write a Bio.PDB structure to a string in PDB format."""
    pdb_io = PDBIO()
    pdb_io.set_structure(structure)
    string_out = io.StringIO()
    pdb_io.save(string_out)
    return string_out.getvalue()

def pdb_to_xyz_string(pdb_text):
    """Convert PDB text to XYZ format string."""
    xyz_lines = []
    atom_count = 0
    
    # Parse PDB lines
    for line in pdb_text.split('\n'):
        if line.startswith('ATOM') or line.startswith('HETATM'):
            atom_count += 1
            atom_type = line[76:78].strip()  # Element symbol
            if not atom_type:  # If element symbol not in columns 76-77, get from atom name
                atom_type = line[12:16].strip()[0]
            x = float(line[30:38])
            y = float(line[38:46]) 
            z = float(line[46:54])
            xyz_lines.append(f"{atom_type} {x:.3f} {y:.3f} {z:.3f}")
    
    # Construct XYZ format
    xyz_text = f"{atom_count}\nConverted from PDB\n" + "\n".join(xyz_lines)
    return xyz_text

# ---------- Main Code ----------

# Load the original PDB file
pdbfile = "alanine_dipeptide_nowater.pdb"
with open(pdbfile, "r") as f:
    original_pdb_text = f.read()

# Parse the structure
parser = PDBParser(QUIET=True)
structure_orig = parser.get_structure("AD", pdbfile)

# Make a deep copy for modifications
structure_mod = copy.deepcopy(structure_orig)

# Set target dihedral angles
delta_phi = 90.0
delta_psi = 0.0
target_phi = -90.0 + delta_phi
target_psi = 127.81 + delta_psi

# Apply modifications on the copied structure
set_phi_dihedral(structure_mod, target_phi)
set_psi_dihedral(structure_mod, target_psi)

# Convert structures to XYZ format
original_xyz_text = pdb_to_xyz_string(original_pdb_text)
modified_xyz_text = pdb_to_xyz_string(structure_to_pdb_string(structure_mod))

# ---------- Visualization using py3Dmol ----------

# Create a viewer with two models
view = py3Dmol.view(width=800, height=400)

# Add the original structure
view.addModel(original_xyz_text, 'xyz')

color_map = {
    "C": "#009D78",   # Carbon (Green)
    "O": "#FF180C",   # Oxygen (Red)
    "N": "#2757DB",   # Nitrogen (Blue)
    "H": "#CCCCCC",   # Hydrogen (White)
}

style = "spheressticks"

if style == "stick":
    # Style the original structure 
    view.setStyle({'model': 0}, {'stick': {'colorscheme': color_map}})

    # Add the modified structure as a second model
    view.addModel(modified_xyz_text, 'xyz')
    view.setStyle({'model': 1}, {'stick': {'colorscheme': color_map}})

    # Set transparency
    view.setStyle({'model': 0}, {'stick': {'opacity': 0.6}})
    view.setStyle({'model': 1}, {'stick': {'opacity': 0.9}})

elif style == "spheressticks":
    # Style the original structure 
    view.setStyle({'model': 0}, {
        'stick': {'colorscheme': color_map, 'radius': 0.2, 'opacity': 0.6},
        "sphere": {"radius": 0.4, "colorscheme": color_map, "opacity": 0.6},
    })

    # Add the modified structure as a second model
    view.addModel(modified_xyz_text, 'xyz')
    view.setStyle({'model': 1}, {
        'stick': {'colorscheme': color_map, 'radius': 0.2, 'opacity': 0.9},
        "sphere": {"radius": 0.4, "colorscheme": color_map, "opacity": 0.9},
    })

else:
    raise ValueError("Invalid style")

view.zoomTo()
view.show()

indices for phi: [('X', 1, 'C'), ('X', 2, 'N'), ('X', 2, 'CA'), ('X', 2, 'C')]
Current φ: -90.00°; rotating by 90.00° to 0.00°
indices for psi: [('X', 2, 'N'), ('X', 2, 'CA'), ('X', 2, 'C'), ('X', 3, 'N')]
Current ψ: 127.81°; rotating by 0.00° to 127.81°


3Dmol.js failed to load for some reason. Please check your browser console for error messages.